In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
from math import sqrt
import pandas_profiling

import env
import acquire
import prep
import explore_final
import scipy.stats as stats
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from statsmodels.formula.api import ols

## Initial Hypothesis

Square footage and home value extremes are leading drivers of log error.

## Acquire and summarize data

Initial Dataframe included single family residence homes with 40+ initial fields brought in.

After reviewing the data for missing values, multicollinearity, and low correlation we dropped our data down to 14 columns and 54,000 rows. 300 values were imputed on lotsizesquarefeet with the mean in order to maintain the column.

In [2]:
#nmysql query single family residence from zillow DB
df = prep.prep_df()

In [10]:
df.head(2)

,sqft,tax_value,latitude,longitude,lotsizesquarefeet,structuretaxvaluedollarcnt,logerror
0,3633.0,296425.0,34560018.0,-118169806.0,9826.000000,222321.0,0.042463
1,1620.0,847770.0,33996200.0,-118438000.0,12842.749037,339532.0,-0.019546


## Data prep and test/train split

Our initial dataframe was split into 70/30 train/test adding a tax_per_sqft field. Random State 123 was used.

In [4]:
#train test split and adding a tax_per_sqft field
train, test = prep.get_train_and_test(df)

sqft                          3.400000e+03
tax_value                     1.621085e+06
latitude                      3.411540e+07
longitude                    -1.184282e+08
lotsizesquarefeet             1.705000e+04
structuretaxvaluedollarcnt    3.242110e+05
logerror                      4.126122e-02
Name: 45911, dtype: float64

In [5]:
train.head(2)

,sqft,tax_value,latitude,longitude,lotsizesquarefeet,structuretaxvaluedollarcnt,logerror,tax_per_sqft
45911,3400.0,1621085.0,34115399.0,-118428166.0,17050.0,324211.0,0.041261,476.789706
11373,2805.0,621455.0,34200510.0,-118920525.0,7067.0,372876.0,-0.409181,221.552585


## Initial exploration and feature selection

In [6]:
#adding a baseline yhat as mean of log error and residual amt based off that.
train['mean_logerror'] = train.logerror.mean()
train['residual'] = train['logerror']- train['mean_logerror']

In [7]:
train.head(2)

,sqft,tax_value,latitude,longitude,lotsizesquarefeet,structuretaxvaluedollarcnt,logerror,tax_per_sqft,mean_logerror,residual
45911,3400.0,1621085.0,34115399.0,-118428166.0,17050.0,324211.0,0.041261,476.789706,0.016716,0.024546
11373,2805.0,621455.0,34200510.0,-118920525.0,7067.0,372876.0,-0.409181,221.552585,0.016716,-0.425896


In [9]:
# Heatmap of possible initial features
plt.figure(figsize=(12,10))
subset = df[['bathrooms','bedrooms','sqft','tax_value','logerror']]
cor = subset.corr()
sns.heatmap(cor,annot=True,cmap=plt.cm.Blues)
plt.show()

KeyError: "['bathrooms', 'bedrooms'] not in index"

<Figure size 864x720 with 0 Axes>

In [ ]:
X_train, y_train, X_test, y_test = prep.get_baseline_train_test_split(df)

In [ ]:
X_train.head(2)

## Initial Baseline score

In [ ]:
baseline = mean_squared_error(train.logerror,train.mean_logerror)
print(baseline)

## MVP

After some basic feature engineering and selection we trained the new model on just sqft and bedrooms. This model performed slightly better than our baseline.

In [ ]:
#MSE of sqft and bedrooms as x variables
x = train[['sqft','bedrooms']]
y = train[['logerror']]
ols_model = ols('y ~ x', data=train).fit()
train['yhat'] = ols_model.predict(x)

In [ ]:
model1 = mean_squared_error(train.logerror,train.yhat)
print(model1)
model1 < baseline

## Exploration

We began our exploration by clustering on the y variable.

In [ ]:
explore_final.elbow_plot(y_train)

In [ ]:
#Clustered by logerror. function takes in y_train,X_train, and num_clusters
train = explore_final.target_cluster(y_train,X_train,6)

In [ ]:
train.groupby('cluster').mean().sort_values(by='logerror')

### Target clustering

The crosstab above shows the clusters sorted by lowest average log error to highest. The summary data shows that the largest/most expensive, and smallest/least expensive houses had the largest absolute value logerrors.

Unfortunately...this was due to our clusters being very disproportionate.

In [ ]:
train.cluster.value_counts()

### Variable clustering



In [ ]:
X_train.head(2).T

In [ ]:
#X_train = train[['tax_per_sqft','structuretaxvaluedollarcnt','lotsizesquarefeet']]
#X_test = test[['tax_per_sqft','structuretaxvaluedollarcnt','lotsizesquarefeet']]